In [22]:
import os
#import numpy as np
import cv2
#import matplotlib.pyplot as plt
import pickle 
import random
import matplotlib.pyplot as plt #library untuk menampilkan gambar
import numpy as np # Library untuk array
from PIL import Image #Library 
from sklearn.metrics import accuracy_score, confusion_matrix #Library untuk menampilkan tabel confussion
from sklearn.model_selection import train_test_split # Library untuk mengacak data
from sklearn.neighbors import KNeighborsClassifier #Library KNN
from sklearn.preprocessing import LabelEncoder #Library labeling
from LocalBinaryPattern import LBP #library LBP

In [23]:
# Mengimport dataset
def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName) # Membuat sebuah array/list untuk menyimpan semua gambar dataset
    allFiles = list() # Kemudian ambil setiap file atau folder simpan kedalam varibel listOfFile
    for entry in listOfFile: # Melakukan perulangan terhadap setiap file atau folder
        # Lakukan pengecekan
        fullPath = os.path.join(dirName, entry) # Mengambil alamat folder data
        # Jika varibel fullpath adalah sebuah folder maka panggil kembali fungsi getListOfFiles()
        # Jika tidak maka akan menambahkan file gambar dataset kedalam varibale allFiles        
        if os.path.isdir(fullPath):# Lakukan pengecekan
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
            # Kemudian ambil setiap file atau folder simpan kedalam varibel listOfFile
    return allFiles #mengambil hasil proses dataset

imagePaths = getListOfFiles("./data/") #mengambil return (variabel all files)

In [24]:
# Proses Labeling
lables = [] #menyimpan label data
for image in imagePaths: #memproses semua data dari imagepaths
    lable = os.path.split(os.path.split(image)[0])[1] 
    # membaca file/folder 0= folder data dan 1 = folder citra,kaget dan madu 
    lables.append(lable) # menyimpan array dari "lables"
lables = np.array(lables) # np.array (array label data yang dapat diolah)
le = LabelEncoder() #library labeling secara data
lables = le.fit_transform(lables) #mengubah citra menjadi label 0,1,dan 2

In [ ]:
data = [] #menyimpan array data
for image in imagePaths: #memproses semua data dari imagepaths

    img = cv2.imread(image) #membaca gambar
    #height, width, _ = img.shape
        
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #proses rgb
    
    img_resize = cv2.resize(img_rgb, (100,200))
    img_gray = cv2.cvtColor(img_resize, cv2.COLOR_RGB2GRAY) #proses rgb ke grayscale

    
    lbp = LBP(); #proses lbp(library)
    img_lbp = lbp.convert(img_gray) #proses grayscale ke lbp
    hist_lbp = cv2.calcHist([img_lbp], [0], None, [256], [0, 256]) #proses histogram lbp
    data.append(hist_lbp) #menyimpan histogram kedalam array "data

In [ ]:
data = np.array(data) # Library untuk dapat menggunakan array

dataset_size = data.shape[0]# menyesuaikan array data set dengan data label
data = data.reshape(dataset_size,-1)# menyesuaikan array data set dengan data label
trainX, testX, trainY, testY = train_test_split(data, lables, test_size=0.2, random_state=17)
#proses pemisahan dataset menjadi datalatih dan datauji, dan random state untuk mengacak data

In [ ]:
knn = KNeighborsClassifier(n_neighbors=2) # Penentuan nilai K
knn.fit(trainX, trainY) # Mengambil nilai dari data latih x dan data latih y  
prediction=knn.predict(testX) # Melakukan proses prediksi dari data uji
acc=accuracy_score(testY, prediction)*100 # Proses perhitungan akurasi
from sklearn.metrics import classification_report # library classificarion report
print(classification_report(testY, prediction)) # Menampilkan tabel atau hasil presisi, recall, akurasi
cm = confusion_matrix(testY, prediction) # Melakukan proses confussion matrix
print('Akurasi:', acc) # Menampilkan kata "akurasi"
print('Confusion Matrix') # Hasil kata "confussion matrix"
print(cm) # Menampilkan matriks confussion matrix

In [ ]:
pickle.dump(knn, open("model.sav", "wb"))

In [ ]:
#print (trainX)
#print (trainY)
#print(testX)
#print(testY)
#print(prediction)

#print(data)

In [ ]:
from sklearn.model_selection import cross_val_score
from numpy import arange

s = list(arange(1, 75, 2))
scores = []
for k in s:
    knn = KNeighborsClassifier(n_neighbors=k)
    score = cross_val_score(knn, trainX, trainY, scoring='accuracy')
    scores.append(score.mean())
mse = [1 - x for x in scores]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(s, mse)
plt.xlabel('K')
plt.ylabel('Mean Squared Error')
plt.show()

In [ ]:
import pandas as pd 
from sklearn.utils import shuffle

df = pd.DataFrame(data)
df['label_kelas'] = lables
coba = shuffle(df)
coba
df.to_csv('datahitung.csv')
coba

In [ ]:
import pandas as pd 
df.to_csv("datahitung.csv")